In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import yaml

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
folder_to_harvest = Path("../data/douhet")

json_paths = folder_to_harvest.glob("**/*.json")

db = pd.DataFrame()

for data_path in json_paths:

    course_label =  data_path.stem.replace("_data", "")
    course, edition, ctx = course_label.split("_")

    associazione_path = Path(f"./data/abgrid/{course}/{course}_{edition}/associazione.xlsx")
    associazione_df = pd.read_excel(associazione_path)
    
    with open(data_path, "r") as fin:
        
        data = json.loads(fin.read())
        root_key = list(data.keys())[0]
        
        sna_micro_stats_a = data[root_key]["sna"]["micro_stats_a"]
        sna_micro_stats_b = data[root_key]["sna"]["micro_stats_b"]
        sna_micro_sociogram = data[root_key]["sociogram"]["micro_stats"]
        
        sna_micro_stats_a_df = pd.DataFrame(sna_micro_stats_a).T.add_suffix("_a", axis=1)
        sna_macro_stats_b_df = pd.DataFrame(sna_micro_stats_b).T.add_suffix("_b", axis=1)
        sociogram_micro_stats_df = pd.DataFrame(sna_micro_sociogram).T
        
        course_df =  pd.concat([sna_micro_stats_a_df, sna_macro_stats_b_df, sociogram_micro_stats_df], axis=1)
        course_df = course_df.reset_index(names="id")
        course_df.insert(0, "corso", course)
        course_df.insert(1, "anno", edition)
        course_df.insert(3, "contesto", ctx)
        
        course_df = course_df.merge(associazione_df, left_on="id", right_on="lettera").drop("lettera", axis=1)
        columns_to_reorder =  list(course_df.columns)
        course_df = course_df.loc[:, [ *columns_to_reorder[:1], columns_to_reorder[-1], *columns_to_reorder[1:-1] ]]
    
    db = pd.concat([db, course_df])

db.soggetto = db.soggetto.str.lower()
db.loc[db.soggetto == "albanese michele", "soggetto"] = "albanese michelle"
db.loc[db.soggetto == "iovane daniele", "soggetto"] = "iovane daniele thomas"
db.loc[db.soggetto == "luongo beatrice", "soggetto"] = "luongo beatrice gaia azzurra"
db.loc[db.soggetto == "panicola giacomo christofer", "soggetto"] = "panicola giacomo christopher"
db.loc[db.soggetto == "suriano campagna", "soggetto"] = "suriano campagna mario"

In [3]:
long_db = db.copy()
long_db = long_db.rename(columns={"soggetto":  "id", "id": "lettera"})
long_db = long_db.filter(items=[c for c in long_db.columns if "rank" not in c])
long_db = long_db.filter(items=[c for c in long_db.columns if "lns" not in c])
c = long_db.columns
long_db = long_db.loc[:, [c[1], c[0], c[2], c[4], c[3], *c[5:] ]]

long_db = long_db.loc[long_db.corso == "vega", :].drop(columns=["corso"])
long_db.head()
long_db.to_excel("./analysis/abgrid_long.xlsx", index=False)

In [4]:
wide_db = long_db.sort_values(by=["id", "anno", "contesto"])
wide_db = wide_db.pivot(index="id", columns=["contesto", "anno"])
wide_db.columns = ['_'.join(col).strip() for col in wide_db.columns.values]
wide_db.reset_index(names=["id"]).to_excel("./analysis/abgrid_wide.xlsx", index=False, na_rep=np.nan)
wide_db

,lettera_lud_i,lettera_stu_i,lettera_lud_ii,lettera_stu_ii,ic_a_lud_i,ic_a_stu_i,ic_a_lud_ii,ic_a_stu_ii,kz_a_lud_i,kz_a_stu_i,kz_a_lud_ii,kz_a_stu_ii,pr_a_lud_i,pr_a_stu_i,pr_a_lud_ii,pr_a_stu_ii,bt_a_lud_i,bt_a_stu_i,bt_a_lud_ii,bt_a_stu_ii,cl_a_lud_i,cl_a_stu_i,cl_a_lud_ii,cl_a_stu_ii,hu_a_lud_i,hu_a_stu_i,hu_a_lud_ii,hu_a_stu_ii,nd_a_lud_i,nd_a_stu_i,nd_a_lud_ii,nd_a_stu_ii,ic_b_lud_i,ic_b_stu_i,ic_b_lud_ii,ic_b_stu_ii,kz_b_lud_i,kz_b_stu_i,kz_b_lud_ii,kz_b_stu_ii,pr_b_lud_i,pr_b_stu_i,pr_b_lud_ii,pr_b_stu_ii,bt_b_lud_i,bt_b_stu_i,bt_b_lud_ii,bt_b_stu_ii,cl_b_lud_i,cl_b_stu_i,cl_b_lud_ii,cl_b_stu_ii,hu_b_lud_i,hu_b_stu_i,hu_b_lud_ii,hu_b_stu_ii,nd_b_lud_i,nd_b_stu_i,nd_b_lud_ii,nd_b_stu_ii,rp_lud_i,rp_stu_i,rp_lud_ii,rp_stu_ii,rr_lud_i,rr_stu_i,rr_lud_ii,rr_stu_ii,gp_lud_i,gp_stu_i,gp_lud_ii,gp_stu_ii,gr_lud_i,gr_stu_i,gr_lud_ii,gr_stu_ii,mp_lud_i,mp_stu_i,mp_lud_ii,mp_stu_ii,mr_lud_i,mr_stu_i,mr_lud_ii,mr_stu_ii,bl_lud_i,bl_stu_i,bl_lud_ii,bl_stu_ii,or_lud_i,or_stu_i,or_lud_ii,or_stu_ii,im_lud_i,im_stu_i,im_lud_ii,im_stu_ii,ai_lud_i,ai_stu_i,ai_lud_ii,ai_stu_ii,ii_lud_i,ii_stu_i,ii_lud_ii,ii_stu_ii,st_lud_i,st_stu_i,st_lud_ii,st_stu_ii
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
abbamondi mario,A,A,A,A,0.081081,0.0,0.107143,0.107143,0.171032,0.111383,0.189548,0.187262,0.042997,0.004934,0.057399,0.061845,0.011887,0.0,0.05291,0.072156,0.303332,0.0,0.3667,0.394366,0.053911,0.017801,0.050918,0.040528,0,1,0,0,0.054054,0.054054,0.035714,0.107143,0.152825,0.157742,0.17405,0.218888,0.018422,0.017352,0.025082,0.106543,0.014264,0.008258,0.0,0.005291,0.054054,0.054054,0.047619,0.251742,0.048975,0.057605,0.0,0.043932,0,0,2,0,3,0,3,3,2,2,1,3,3,3,3,3,3,3,0,1,2,0,2,1,0,0,0,1,1,-2,2,0,0,0,3,2,5,2,4,6,1,-2,5,2,5,0,5,4,ambitendent,disliked,appreciated,ambitendent
albanese michelle,B,B,B,B,0.027027,0.027027,0.178571,0.035714,0.131968,0.140941,0.214826,0.141925,0.009828,0.042179,0.038896,0.013434,0.021396,0.141479,0.150551,0.039638,0.061776,0.335949,0.388593,0.247788,0.01565,0.01975,0.009917,0.026952,0,0,0,0,0.0,0.027027,0.035714,0.0,0.127355,0.154362,0.182663,0.153288,0.008635,0.029406,0.042476,0.009501,0.0,0.005255,0.033069,0.0,0.0,0.204247,0.194444,0.0,0.032719,0.081605,0.081769,0.065713,1,0,0,1,1,1,5,1,0,1,1,0,3,3,3,3,3,3,3,3,1,0,2,1,0,1,1,0,1,0,4,1,0,0,0,0,1,2,6,1,1,0,4,1,2,1,7,2,marginal,ambitendent,appreciated,marginal
bagatin giulio,C,C,NaN,NaN,0.081081,0.054054,NaN,NaN,0.161841,0.137366,NaN,NaN,0.02781,0.011243,NaN,NaN,0.050075,0.014802,NaN,NaN,0.284029,0.202878,NaN,NaN,0.080617,0.025469,NaN,NaN,0,0,NaN,NaN,0.0,0.0,NaN,NaN,0.127355,0.131452,NaN,NaN,0.008635,0.011076,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.047432,0.018157,NaN,NaN,1,1,NaN,NaN,3,2,NaN,NaN,0,0,NaN,NaN,3,3,NaN,NaN,3,3,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,3,2,NaN,NaN,0,0,NaN,NaN,3,2,NaN,NaN,3,2,NaN,NaN,3,2,NaN,NaN,appreciated,appreciated,NaN,NaN
bonfanti francesca,D,D,C,C,0.027027,0.378378,0.107143,0.25,0.128029,0.297547,0.168415,0.226456,0.013109,0.05173,0.012132,0.024796,0.017267,0.091629,0.036243,0.109832,0.027027,0.395484,0.185969,0.261682,0.005962,0.042549,0.02521,0.005156,0,0,0,0,0.0,0.027027,0.178571,0.0,0.127355,0.146043,0.237789,0.153288,0.008635,0.015355,0.05388,0.009501,0.0,0.000751,0.0,0.0,0.0,0.040541,0.206044,0.0,0.0,0.001682,0.0,0.060143,3,0,2,1,1,14,3,7,0,1,5,0,3,3,3,3,0,2,0,3,1,1,0,2,0,0,0,0,1,13,-2,7,3,1,3,0,1,15,8,7,4,14,1,7,2,15,3,9,marginal,popular,disliked,appreciated
ciliento giacomo pio,E,E,D,D,0.135135,0.081081,0.214286,0.285714,0.193306,0.162879,0.238072,0.282277,0.040211,0.038719,0.067044,0.110191,0.050663,0.022472,0.10377,0.087645,0.284029,0.343332,0.419931,0.538462,0.019668,0.039374,0.049236,0.042894,0,0,0,0,0.0,0.0,0.0,0.035714,0.127355,0.131452,0.156802,0.170798,0.008635,0.011076,0.018394,0.015207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128571,0.0,0.0,0.0,0.0,3,3,3,2,5,3,6,8,0,0,0,1,3,3,3,3,0,0,0,0,1,0,2,1,0,0,0,0,5,3,6,7,3,3,3,3,5,3,6,9,8,6,9,10,6,3,8,9,popular,appreciated,appreciated,ap

In [5]:
wide_db[wide_db.isna().sum(axis=1)>0].reset_index(names="id").loc[:, ["id"]].to_excel("./analysis/casi_da_attenzionare.xlsx", index=False)